# Functions

## What are functions?

Functions are computational devices that transform input _values_ into output _values_, and do nothing _else_.

In [ ]:
// `add one` function



If we run this function, the only thing that happens is the computation of a new value:

Functions that do something else, besides returning values, are called _impure_ functions. Functional programming deals only with _pure_, or mathematical, functions.

In [ ]:
// An impure function
/*
def impureAdd(input: Int): Int = {
    input + 1
}
*/

If we run this function, we will see an _effect_ in the console (besides the pure computation of `input + 1`): 

There are many kinds of effects: writing to the console, reading from the keyworkd, reading from a socket, calling a web service, executing a query over the database, etc. Clearly, we need effects if we want our programs to do something useful, so pure functions alone are not enough. We will talk about this later on.



## Functions as modularity devices

Why are functions so important in programming? Because they help us to _modularize_ our code. For instance, let's consider the following programs, which access the following data structure of key-value pairs (we will talk about this structure in detail later on):

In [ ]:
val config: Map[String, String] = 
    Map("URL" -> "http://hablapps.com",
        "PORT" -> "8080")

Our first program access the configuration data for the value of the "URL" key. If it's not found, then the default value "default.url" is returned (similarly, we will discuss the `match` keyword further in the course).

In [ ]:
// Program 1
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}

Our second program accesses the configuration data for the value of the "PORT" key. If it's not found, then the default value "8080" is returned.

In [ ]:
// Program 2
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}

These two programs do _almost_ the same. The only differences lie in the particular keys and default values the programs refer to, but, otherwise, they do the same thing. However, this _common factor_ is not reflected in the code. Indeed, we may get one program from the other by copy-pasting, a clear signal of [code-smell](https://en.wikipedia.org/wiki/Code_smell).

These programs are _monolythic_, in the sense that they are not made by composing large enough modules. In this case, the common logic of the program and the values it operates on are intermingled in the same code. 

How can we abstract away the differences and package the common logic in a single module? With functions:

In [ ]:
/*
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}
*/

This is an abstract module which we can combine with other modules to get back the very same functionality:

In [ ]:
// Program 1
// val url: String = ???

In this case, we combine the module `getKeyFrom` with the modules (data values and variables, in particular) `config`, `"URL"` and `"default.url"`. The composition method is just simple function application.

Which are the advantages of using functions? As in the general case, having a more modular solution enables _reuse_, particularly of those modules which are abstract or parameterised. For instance, we can benefit from this level of reuse by re-implementing the `url` program in the following way:

In [ ]:
// Program 2
// val port: String = ???

## Functions as methods

In an object-oriented language, functions are implemented through _methods_, i.e. using the `def` keyword. Note that these methods are invariably part of an `object`, `class` or `trait` declaration. Typically, pure functions are declared as part of objects. For instance, we may declare a set of arithmetic functions as follows: 

In [ ]:
import scala.math.{pow, Pi}

object Areas{
    
    def circle(radius: Double): Double = 
        ???
    
    def rectangle(width: Double, height: Double): Double = 
        ???
}

In notebooks and the Scala REPL, `def` declarations appear to be independent from any object or class, but they are not:

In [ ]:
def foo(i: Int): Int = i
// show errors: "missing argument list for method foo in class Helper"


When we study higher-order functions, we will see that functions in Scala can also be represented as _objects_, i.e. not only as methods. However, that representation also builds essentially upon methods.

## Functions as values

Functions can also be represented as _values_, i.e. as objects. This allows us to implement functions that receive other functions as arguments, or return functions as results. This special functions are called _higher-order functions_ (HOF), and they feature as a great modularity device. We will mainly discuss this feature of HOFs in PF-3; now, we just want to focus on how are functions actually represented as values in a OO language like Scala. 

This representation builds essentially upon methods, in particular, _reified_ methods. For instance, let's consider the following functions:

In [16]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

We want to implement a HOF that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and calls this function over a given number. We may want to write something like this:

In [ ]:
// def call(def int2int(n: Int): Int, number: Int): Int =
//   int2int(number)

where the first argument `int2int` attempts to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

In [ ]:
// Reify and abstract the implementation!
/*
def addOneM(number: Int): Int = 
    number + 1
*/

Now, we can implement the `call` HOF as follows: 

In [ ]:
// def call(def int2int(n: Int): Int, number: Int): Int =
//   int2int(number)

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [ ]:
/*
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 
*/

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [1]:
def empty(s: String): Boolean = 
    s == ""

defined function empty

In [2]:
trait FunctionStringToBoolean{
    def apply(s: String): Boolean 
}

defined trait FunctionStringToBoolean

In [3]:
object emptyV extends FunctionStringToBoolean{
    def apply(s: String): Boolean = 
        s == ""
}

defined object emptyV

## Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), etc., roughly implemented as follows:

In [ ]:
object Std{
    
    trait Function1[A, B]{
        def apply(x: A): B
    }
    
    trait Function2[A, B, C]{
        def apply(x: A, y: B): C
    }

    // up to Function22
}


Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [4]:

val addOneV: Function1[Int, Int] = new Function1[Int, Int]{
    def apply(number: Int): Int = 
        number + 1
}

addOneV: Int => Int = <function1>

In [5]:
val emptyV: Function1[String, Boolean] = new Function1[String, Boolean]{
    def apply(x: String): Boolean = 
        x == ""
}

emptyV: String => Boolean = <function1>

In [9]:
emptyV.apply("")
emptyV("")
empty("hola")

res8_0: Boolean = true
res8_1: Boolean = true
res8_2: Boolean = false

But we can do it more easily, since Scala also provides special syntax to declare function types and instantiate functions (so-called _lambda expressions_):

In [10]:

val addOneV: Int => Int = new (Int => Int){
    def apply(a: Int): Int = 
        a + 1
}


// val substractOneV

addOneV: Int => Int = <function1>

In [11]:

val addOneV: Int => Int = 
    (a: Int) => a + 1


// val substractOneV

addOneV: Int => Int = ammonite.$sess.cmd10$Helper$$Lambda$2139/1673914146@67c34f46

And we can also profit from type inference:

In [12]:
val addOneV: Int => Int = 
    a => a + 1

addOneV: Int => Int = ammonite.$sess.cmd11$Helper$$Lambda$2143/311418964@50099827

Using these syntactic facilities, the `call` HOF has a more appealing signature: 

In [13]:

def call(int2int: Int => Int, number: Int): Int = 
    int2int(number)


defined function call

which we can use as follows:

In [17]:
call(addOneV, 5)
call((x: Int) => x + 1, 5)
call(new Function1[Int, Int]{ def apply(x: Int) = x + 1}, 5)

res16_0: Int = 6
res16_1: Int = 6
res16_2: Int = 6

And we can even pass function-methods that are converted on the fly to function-values!

In [20]:
call(addOneM, 6)
call((x: Int) => addOneM(x), 6)

res19_0: Int = 7
res19_1: Int = 7

This is the so-called _eta-expansion_.

Last, we can get extra level of conciseness using so-called _underscore_ syntax:

In [ ]:

val addOneV: Int => Int = 
    _ + 1


In [ ]:

val times: (Int, Int) => Int = 
    _ + _


In [24]:

val times = 
    (a: String, b: String) => a + b


times: (String, String) => String = ammonite.$sess.cmd23$Helper$$Lambda$2213/450651321@3d8ba53a

In [25]:

val times = 
    (_: String) + (_: String)


times: (String, String) => String = ammonite.$sess.cmd24$Helper$$Lambda$2218/67895245@11eac922

In [23]:
times("hol", "a")

res22: String = "hola"

In [28]:

call(_+1, 5) // 5+1


res27: Int = 6

In [30]:
def foo2(x: Int, y: Int): Int = 
    x + y*y // x*y + y*y

defined function foo2

In [31]:
val foo2: (Int, Int, Int) => Int = 
    _ + _ * _

foo2: (Int, Int, Int) => Int = ammonite.$sess.cmd30$Helper$$Lambda$2246/947581303@72b0fbe8

In [31]:
/*
val foo2: (Int, Int) => Int = 
    (_ + _) * _
    */

cmd31.sc:2: missing parameter type for expanded function ((<x$3: error>) => ((<x$1: error>, <x$2: error>) => x$1.$plus(x$2)).<$times: error>(x$3))
    (_ + _) * _
              ^Compilation Failed

: 

In [61]:

val foo2: (Int, Int) => Int = 
    (_ + _) * _


cmd61.sc:2: missing parameter type for expanded function ((<x$3: error>) => ((<x$1: error>, <x$2: error>) => x$1.$plus(x$2)).<$times: error>(x$3))
    (_ + _) * _
              ^Compilation Failed

: 

In [61]:

val foo2 = 
    (c: Int) => ((a: Int, b: Int) => a + b).*(c)


cmd61.sc:2: value * is not a member of (Int, Int) => Int
    (c: Int) => ((a: Int, b: Int) => a + b).*(c)
                                            ^Compilation Failed

: 

In [63]:

val foo2 = 
    (c: Int) => ((a: Int, b: Int) => a + b).apply(c,1)


foo2: Int => Int = ammonite.$sess.cmd62$Helper$$Lambda$2709/1319450873@33458adb

In [64]:

val foo2: Int => Int = 
    ((_: Int) + (_: Int)).apply(_,1)


foo2: Int => Int = ammonite.$sess.cmd63$Helper$$Lambda$2713/14576780@ea9ba46

In [65]:

val foo2: Int => Int = 
    ((_ + _): (Int, Int) => Int).apply(_,1)


foo2: Int => Int = ammonite.$sess.cmd64$Helper$$Lambda$2717/1611264050@3117409f

In [ ]:
def callM(int2int: Int => Int, number: Int): Int = 
    int2int(number)


In [33]:
val callV: Function2[Int => Int /* Function1[Int, Int]*/, Int, Int] = 
    new Function2[Int => Int, Int, Int]{
        def apply(int2int: Int => Int, number: Int): Int = 
            int2int(number)
    }

callV: (Int => Int, Int) => Int = <function2>

In [34]:
val callV: (Int => Int, Int) => Int = 
    (int2int: Int => Int, number: Int) => 
            int2int(number)

callV: (Int => Int, Int) => Int = ammonite.$sess.cmd33$Helper$$Lambda$2507/1291878887@479397e5

In [34]:
val callV: (Int => Int, Int) => Int = 
    (int2int, number) => 
            int2int(number)

callV: (Int => Int, Int) => Int = ammonite.$sess.cmd33$Helper$$Lambda$2507/1291878887@479397e5

In [35]:
val callV: (Int => Int, Int) => Int =  
    _(_)

callV: (Int => Int, Int) => Int = ammonite.$sess.cmd34$Helper$$Lambda$2513/2058555314@489fa35

## Currying

We can use a similar syntax for functions of two arguments. So, the function-value equivalent of this function-method: 

In [59]:
def sumM(x: Int, y: Int): Int = 
    x+y

defined function sumM

could be written in this way: 

In [36]:
val sumV: (Int, Int) => Int = 
    (a, b) => a+b

sumV: (Int, Int) => Int = ammonite.$sess.cmd35$Helper$$Lambda$2519/1471581504@4f8e9007

but also in the following one:

In [ ]:
// val sumV with sugar

or, exploiting type inference:

However, `Function2`, `Function3`, ... classes are not extrictly necessary, and sometimes we can get along with `Function1`. But, how can we create a function of two arguments with `Function1` alone? The trick is the following:

In [41]:

val sumC: Int => Int => Int = 
    // (a: Int, b: Int) => a + b
    (a: Int) => 
        ((b: Int) => a+b : Int) : (Int => Int)


sumC: Int => Int => Int = ammonite.$sess.cmd40$Helper$$Lambda$2551/269917500@5e7aec84

In [43]:
val foo: String => Boolean = _ == ""

foo: String => Boolean = ammonite.$sess.cmd42$Helper$$Lambda$2560/1873419070@715e9f6e

In [ ]:
foo("hola"): Boolean

In [44]:
val suma3: Int => Int = sumC(3): Int => Int

suma3: Int => Int = ammonite.$sess.cmd40$Helper$$Lambda$2556/324831546@6305905

In [47]:
suma3(8)
sumC(3)(8)

res46_0: Int = 11
res46_1: Int = 11

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

returns a function that can be applied again:

In [48]:
def foo(x: String, y: Boolean, c: Char, d: Integer): Double = 
    1.0

defined function foo

In [51]:
val fooV: String => (Boolean => (Char => (Integer => Double))) = 
    _ => _ => _ => _ => 1.0: Double

fooV: String => Boolean => Char => Integer => Double = ammonite.$sess.cmd50$Helper$$Lambda$2611/520602488@61b59c9b

We can apply this strategy to functions of any number of arguments. This is called _currying_ and _currified functions_. The analog in function-methods is [multiple-parameter lists](https://docs.scala-lang.org/tour/multiple-parameter-lists.html):

In [57]:

def sumMC(x: Int)(y: Int): Int = 
    x+y


defined function sumMC

In [55]:
sum(1): (Int => Int)
val f: Int => Int = sum(1)

res54_0: Int => Int = ammonite.$sess.cmd54$Helper$$Lambda$2666/483892394@67a87d37
f: Int => Int = ammonite.$sess.cmd54$Helper$$Lambda$2667/2070113663@1d156fad

In [56]:
sum(1)_

res55: Int => Int = ammonite.$sess.cmd55$Helper$$Lambda$2674/1078043506@40862111

In [58]:
val f: Int => Int = sumMC(1)

f: Int => Int = ammonite.$sess.cmd57$Helper$$Lambda$2678/1825092453@14df6b49

In [60]:
val f: Int => Int = 
    (a: Int) => sumM(1,a): Int

f: Int => Int = ammonite.$sess.cmd59$Helper$$Lambda$2682/880403185@7b276d01

In [61]:
val f: Int => Int = 
    sumM(1,_)

f: Int => Int = ammonite.$sess.cmd60$Helper$$Lambda$2686/452029220@43527a82

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [66]:
def isEvenLength: String => Boolean = 
    (x: String) => x.length % 2 == 0

defined function isEvenLength

In [69]:
isEvenLength("kk")

res68: Boolean = true

This function is somehow the combination of two more basic functions `length` and `isEven`:

In [70]:
val length: String => Int = 
    _.length

length: String => Int = ammonite.$sess.cmd69$Helper$$Lambda$2728/338947084@5a8de9a5

In [71]:
val isEven: Int => Boolean = 
    _ % 2 == 0

isEven: Int => Boolean = ammonite.$sess.cmd70$Helper$$Lambda$2732/462544150@646b7915

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` using the functions `length` and `isEven`? We can use a HOF which helps us to compose functions:

In [72]:
def isEvenLength: String => Boolean = 
    (x: String) => isEven(length(x))

defined function isEvenLength

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [80]:
def compose(f: Int => Boolean, g: String => Int): String => Boolean =
    (s: String) => f(g(s): Int) : Boolean

defined function compose

In [80]:
def compose(f: Int => Boolean, g: String => Int): String => Boolean =
    s => f(g(s))

defined function compose

In [81]:
def compose[A, B, C](f: B => C, g: A => B): A => C =
    s => f(g(s))

defined function compose

In [83]:
def composeV[A, B, C]: (B => C, A => B) => (A => C) =
    //s => f(g(s))
    (f: B => C, g: A => B) => (a: A) => f(g(a : A) : B) : C

defined function composeV

In [80]:
// placeholder syntax doesn't work here
//def compose(f: Int => Boolean, g: String => Int): String => Boolean =
  //  f(g(_))

In [73]:
val isEvenLength: String => Boolean =
    isEven compose length
    // (isEven: Int => Boolean).compose(length: String => Int)

isEvenLength: String => Boolean = scala.Function1$$Lambda$2737/225143494@74ebc2e1

The HOF `compose` is actually defined by `Function1`: 

In [ ]:
// val isEvenLength: String => Boolean = ???

or using infix notation:

In [ ]:
// val isEvenLength: String => Boolean = ???

Note that a similar function to `compose`, called `andThen`, is also available in the standard library: 

In [77]:
val isEvenLength: String => Boolean =
    (length: String => Int) andThen (isEven: Int => Boolean)

isEvenLength: String => Boolean = scala.Function1$$Lambda$257/589311950@1e88f7f0

In [75]:
isEvenLength("hola")

res74: Boolean = true

We can also give a currified version of this function as follows:

In [ ]:
/*
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    (a: A) => f2(f1(a))
*/

Last, there is a function which behaves as the identity element with respect to the operation `compose`, i.e. no matter which other function we choose to compose with the `identity` function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

In [87]:
def identity[A](a: A): A = 
    "".asInstanceOf[A] : A

cmd87.sc:2: type mismatch;
 found   : String("")
 required: A
    "" : A
    ^Compilation Failed

: 

In [86]:
identity[String]("hola")

res85: String = ""

In [87]:
identity[Int](1)

: 

In [ ]:
def identity(s: String): String = 
    "" // "a", "ab", ... , s

or using lambda expressions:

In [88]:
def identity[A](a: A): A = 
    a

defined function identity

In [89]:
def identity[A]: A => A = 
    (a: A) => a : A

defined function identity

## Contravariance and variance in Function traits

Trait `Function1` is actually defined as follows:

In [ ]:
object StdFunctions{
    /*
    trait Function1[A, B]{
        def apply(a: A): B
    }
    */
}

Given the following hierarchy:

In [ ]:
abstract class Animal
class Perro extends Animal
class Doberman extends Perro
class Gato extends Animal
class Siames extends Gato

In [ ]:
def fGP: Gato => Perro = ??? 
def fGD: Gato => Doberman = ??? 
def fGA: Gato => Animal = ???
def fAP: Animal => Perro = ???
def fSP: Siames => Perro = ??? 
def fAD: Animal => Doberman = ???

Which functions can be passed to this method?

In [ ]:
def f2(f: Gato => Perro) = ???

In [ ]:
// def r1 = f2(fGP)
// def r2 = f2(fGD)
// def r3 = f2(fGA)
// def r4 = f2(fAP)
// def r5 = f2(fSP)
// def r6 = f2(fAD)